In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
!ls -l "/content/drive/MyDrive/"

total 133829
drwx------ 5 root root      4096 Jun  2  2025  Classroom
-rw------- 1 root root       183 Nov 13 21:44 'CO7 TECHNOLOGIES: INFORMATION-SYSTEMS SELECTION.gdoc'
drwx------ 2 root root      4096 Feb 24  2025 'Colab Notebooks'
drwx------ 4 root root      4096 Nov  1 15:29 'Maastricht university'
drwx------ 5 root root      4096 Dec 30 19:25 'ML2 regression'
drwx------ 2 root root      4096 Nov  1 15:35  Private
drwx------ 6 root root      4096 Nov  1 15:31 'University of Warsaw'
-rw------- 1 root root 137010832 Dec 26 19:01  USA_housing_project.zip
drwx------ 3 root root      4096 Dec 21 22:29  usa_rent_ml2_regression


In [21]:
!ls -l "/content/drive/MyDrive/ML2 regression"

total 95
drwx------ 4 root root  4096 Dec 30 19:25 artifacts
drwx------ 4 root root  4096 Dec 30 19:25 data
-rw------- 1 root root 84972 Dec 28 18:20 just.docx
drwx------ 3 root root  4096 Dec 30 19:25 notebooks


In [23]:
import os

BASE_PATH = "/content/drive/MyDrive/ML2 regression"
DATA_PATH = f"{BASE_PATH}/data"
MODEL_PATH = f"{BASE_PATH}/artifacts/models"

print("Base path:", BASE_PATH)
print("Data path:", DATA_PATH)
print("Model path:", MODEL_PATH)

# Check folders exist
print("\nContents of BASE_PATH:")
!ls -l "$BASE_PATH"


Base path: /content/drive/MyDrive/ML2 regression
Data path: /content/drive/MyDrive/ML2 regression/data
Model path: /content/drive/MyDrive/ML2 regression/artifacts/models

Contents of BASE_PATH:
total 95
drwx------ 4 root root  4096 Dec 30 19:25 artifacts
drwx------ 4 root root  4096 Dec 30 19:25 data
-rw------- 1 root root 84972 Dec 28 18:20 just.docx
drwx------ 3 root root  4096 Dec 30 19:25 notebooks


In [25]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 4.1 MB/s eta 0:00:00


In [26]:
import os
import pickle
import numpy as np
import pandas as pd
import category_encoders

In [27]:
DATA_PATH = os.path.join(BASE_PATH, "data", "processed")
PREP_PATH = os.path.join(BASE_PATH, "artifacts", "preprocessing")
X_train = pd.read_csv(os.path.join(DATA_PATH, "X_train_raw.csv"))
y_train = pd.read_csv(os.path.join(DATA_PATH, "y_train.csv"))
with open(os.path.join(PREP_PATH, "preprocessor.pkl"), "rb") as f:
    preprocessor = pickle.load(f)
X_train_transformed = np.load(os.path.join(PREP_PATH, "X_train_transformed.npy"))

print("Training artifacts loaded successfully.")

Training artifacts loaded successfully.


In [28]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pickle
import os

In [29]:
y_train_series = np.ravel(y_train)
best_params_dt = {
    "criterion": "friedman_mse",
    "splitter": "best",
    "max_depth": 11,
    "min_samples_split": 8,
    "min_samples_leaf": 9,
    "max_features": None,
    "ccp_alpha": 0.00022707289534838138,
    "random_state": 42
}

dt_best = DecisionTreeRegressor(**best_params_dt)
dt_best.fit(X_train_transformed, y_train_series)
y_pred_dt = dt_best.predict(X_train_transformed)
rmse_dt = np.sqrt(mean_squared_error(y_train_series, y_pred_dt))
mae_dt = mean_absolute_error(y_train_series, y_pred_dt)
r2_dt = r2_score(y_train_series, y_pred_dt)
mape_dt = np.mean(np.abs((y_train_series - y_pred_dt) / (y_train_series + 1e-8))) * 100

print("\n" + "="*50)
print("Decision Tree (Best Params) — Training Results")
print("="*50)
print(f"RMSE : {rmse_dt:.4f}")
print(f"MAE  : {mae_dt:.4f}")
print(f"MAPE : {mape_dt:.2f}%")
print(f"R²   : {r2_dt:.4f}")
print("="*50)


Decision Tree (Best Params) — Training Results
RMSE : 0.3570
MAE  : 0.2129
MAPE : 4.03%
R²   : 0.5018


In [30]:
MODEL_PATH = f"{BASE_PATH}/artifacts/models"
os.makedirs(MODEL_PATH, exist_ok=True)

save_path = os.path.join(MODEL_PATH, "decision_tree_best.pkl")
with open(save_path, "wb") as f:
    pickle.dump(dt_best, f)

print(f"\nModel saved to: {save_path}")


Model saved to: /content/drive/MyDrive/ML2 regression/artifacts/models/decision_tree_best.pkl


In [31]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pickle
import os

In [32]:
y_train_series = np.ravel(y_train)
best_params_xgb = {
    "n_estimators": 500,
    "learning_rate": 0.08169314570885453,
    "max_depth": 11,
    "subsample": 0.8493192507310232,
    "colsample_bytree": 0.9085081386743783,
    "gamma": 0.3702232586704518,
    "min_child_weight": 7,
    "objective": "reg:squarederror",
    "tree_method": "hist",
    "random_state": 42,
    "n_jobs": -1
}

xgb_best = XGBRegressor(**best_params_xgb)
xgb_best.fit(X_train_transformed, y_train_series)
y_pred_xgb = xgb_best.predict(X_train_transformed)

rmse_xgb = np.sqrt(mean_squared_error(y_train_series, y_pred_xgb))
mae_xgb = mean_absolute_error(y_train_series, y_pred_xgb)
r2_xgb = r2_score(y_train_series, y_pred_xgb)
mape_xgb = np.mean(np.abs((y_train_series - y_pred_xgb) / (y_train_series + 1e-8))) * 100
# 5. Print results
# ============================
print("\n" + "="*50)
print("XGBoost (Best Params) — Training Results")
print("="*50)
print(f"RMSE : {rmse_xgb:.4f}")
print(f"MAE  : {mae_xgb:.4f}")
print(f"MAPE : {mape_xgb:.2f}%")
print(f"R²   : {r2_xgb:.4f}")
print("="*50)


XGBoost (Best Params) — Training Results
RMSE : 0.2004
MAE  : 0.1071
MAPE : 2.00%
R²   : 0.8430


In [33]:
MODEL_PATH = f"{BASE_PATH}/artifacts/models"
os.makedirs(MODEL_PATH, exist_ok=True)

save_path = os.path.join(MODEL_PATH, "xgboost_best.pkl")
with open(save_path, "wb") as f:
    pickle.dump(xgb_best, f)

print(f"\nModel saved to: {save_path}")


Model saved to: /content/drive/MyDrive/ML2 regression/artifacts/models/xgboost_best.pkl


In [35]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 6.0 MB/s eta 0:00:00


In [36]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pickle
import os

In [37]:
y_train_series = np.ravel(y_train)

best_params_cat = {
    "depth": 10,
    "learning_rate": 0.08169314570885453,   # same LR as XGB search result
    "iterations": 1200,
    "l2_leaf_reg": 1.0,
    "bagging_temperature": 0.0,
    "random_strength": 0.0,
    "loss_function": "RMSE",
    "random_seed": 42,
    "verbose": 0
}

cat_best = CatBoostRegressor(**best_params_cat)
cat_best.fit(X_train_transformed, y_train_series)
y_pred_cat = cat_best.predict(X_train_transformed)
rmse_cat = np.sqrt(mean_squared_error(y_train_series, y_pred_cat))
mae_cat = mean_absolute_error(y_train_series, y_pred_cat)
r2_cat = r2_score(y_train_series, y_pred_cat)
mape_cat = np.mean(np.abs((y_train_series - y_pred_cat) / (y_train_series + 1e-8))) * 100
print("\n" + "="*50)
print("CatBoost (Best Params) — Training Results")
print("="*50)
print(f"RMSE : {rmse_cat:.4f}")
print(f"MAE  : {mae_cat:.4f}")
print(f"MAPE : {mape_cat:.2f}%")
print(f"R²   : {r2_cat:.4f}")
print("="*50)


CatBoost (Best Params) — Training Results
RMSE : 0.1738
MAE  : 0.0971
MAPE : 1.71%
R²   : 0.8819


In [38]:
MODEL_PATH = f"{BASE_PATH}/artifacts/models"
os.makedirs(MODEL_PATH, exist_ok=True)

save_path = os.path.join(MODEL_PATH, "catboost_strong.pkl")
with open(save_path, "wb") as f:
    pickle.dump(cat_best, f)

print(f"\nModel saved to: {save_path}")


Model saved to: /content/drive/MyDrive/ML2 regression/artifacts/models/catboost_strong.pkl


In [39]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pickle
import os

In [40]:
y_train_series = np.ravel(y_train)
best_params_rf = {
    "bootstrap": True,
    "max_depth": 16,
    "max_features": "sqrt",
    "min_samples_leaf": 1,
    "min_samples_split": 13,
    "n_estimators": 513,
    "random_state": 42,
    "n_jobs": -1
}

rf_best = RandomForestRegressor(**best_params_rf)
rf_best.fit(X_train_transformed, y_train_series)
y_pred_rf = rf_best.predict(X_train_transformed)

rmse_rf = np.sqrt(mean_squared_error(y_train_series, y_pred_rf))
mae_rf = mean_absolute_error(y_train_series, y_pred_rf)
r2_rf = r2_score(y_train_series, y_pred_rf)
mape_rf = np.mean(np.abs((y_train_series - y_pred_rf) / (y_train_series + 1e-8))) * 100

print("\n" + "="*50)
print("Random Forest (Best Params) — Training Results")
print("="*50)
print(f"RMSE : {rmse_rf:.4f}")
print(f"MAE  : {mae_rf:.4f}")
print(f"MAPE : {mape_rf:.2f}%")
print(f"R²   : {r2_rf:.4f}")
print("="*50)


Random Forest (Best Params) — Training Results
RMSE : 0.2916
MAE  : 0.1648
MAPE : 3.34%
R²   : 0.6676


In [41]:
MODEL_PATH = f"{BASE_PATH}/artifacts/models"
os.makedirs(MODEL_PATH, exist_ok=True)

save_path = os.path.join(MODEL_PATH, "random_forest_best.pkl")
with open(save_path, "wb") as f:
    pickle.dump(rf_best, f)

print(f"\nModel saved to: {save_path}")


Model saved to: /content/drive/MyDrive/ML2 regression/artifacts/models/random_forest_best.pkl


In [42]:
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pickle
import os

In [43]:
y_train_series = np.ravel(y_train)
bag_default = BaggingRegressor(
    n_estimators=50,
    random_state=42,
    n_jobs=-1
)

bag_default.fit(X_train_transformed, y_train_series)
y_pred_bag = bag_default.predict(X_train_transformed)
rmse = np.sqrt(mean_squared_error(y_train_series, y_pred_bag))
mae = mean_absolute_error(y_train_series, y_pred_bag)
r2 = r2_score(y_train_series, y_pred_bag)
mape = np.mean(np.abs((y_train_series - y_pred_bag) / (y_train_series + 1e-8))) * 100

print("\n" + "="*50)
print("Bagging (Default) — Training Results")
print("="*50)
print(f"RMSE : {rmse:.4f}")
print(f"MAE  : {mae:.4f}")
print(f"MAPE : {mape:.2f}%")
print(f"R²   : {r2:.4f}")
print("="*50)


Bagging (Default) — Training Results
RMSE : 0.0978
MAE  : 0.0314
MAPE : 0.61%
R²   : 0.9626


In [44]:
MODEL_PATH = f"{BASE_PATH}/artifacts/models"
os.makedirs(MODEL_PATH, exist_ok=True)

save_path = os.path.join(MODEL_PATH, "bagging_model_best.pkl")
with open(save_path, "wb") as f:
    pickle.dump(bag_default, f)

print(f"\nModel saved to: {save_path}")


Model saved to: /content/drive/MyDrive/ML2 regression/artifacts/models/bagging_model_best.pkl


In [45]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pickle
import os

In [46]:
y_train_series = np.ravel(y_train)
base_tree = DecisionTreeRegressor(
    max_depth=14,
    min_samples_split=13,
    min_samples_leaf=3,
    random_state=42
)
bag_tuned = BaggingRegressor(
    estimator=base_tree,
    n_estimators=32,
    max_samples=0.7962072844310213,
    max_features=0.7571172192068059,
    bootstrap=True,
    random_state=42,
    n_jobs=-1
)

bag_tuned.fit(X_train_transformed, y_train_series)
y_pred_bag_tuned = bag_tuned.predict(X_train_transformed)
rmse = np.sqrt(mean_squared_error(y_train_series, y_pred_bag_tuned))
mae = mean_absolute_error(y_train_series, y_pred_bag_tuned)
r2 = r2_score(y_train_series, y_pred_bag_tuned)
mape = np.mean(np.abs((y_train_series - y_pred_bag_tuned) / (y_train_series + 1e-8))) * 100

print("\n" + "="*50)
print("Bagging (Tuned) — Training Results")
print("="*50)
print(f"RMSE : {rmse:.4f}")
print(f"MAE  : {mae:.4f}")
print(f"MAPE : {mape:.2f}%")
print(f"R²   : {r2:.4f}")
print("="*50)


Bagging (Tuned) — Training Results
RMSE : 0.2598
MAE  : 0.1449
MAPE : 2.80%
R²   : 0.7362


In [47]:
MODEL_PATH = f"{BASE_PATH}/artifacts/models"
os.makedirs(MODEL_PATH, exist_ok=True)

save_path = os.path.join(MODEL_PATH, "bagging_model_tuned.pkl")
with open(save_path, "wb") as f:
    pickle.dump(bag_tuned, f)

print(f"\nModel saved to: {save_path}")


Model saved to: /content/drive/MyDrive/ML2 regression/artifacts/models/bagging_model_tuned.pkl


In [48]:
import numpy as np
import pickle
import os

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [49]:
MODEL_PATH = f"{BASE_PATH}/artifacts/models"

def load_model(name):
    with open(os.path.join(MODEL_PATH, name), "rb") as f:
        return pickle.load(f)

print("Loading base models...")

dt_model       = load_model("decision_tree_best.pkl")
rf_model       = load_model("random_forest_best.pkl")
bag_default    = load_model("bagging_model_best.pkl")
bag_tuned      = load_model("bagging_model_tuned.pkl")
xgb_model      = load_model("xgboost_best.pkl")
cat_model      = load_model("catboost_strong.pkl")

print("All base models loaded successfully.")
y_train_series = np.ravel(y_train)
print("Generating meta-features...")

meta_features = np.column_stack([
    dt_model.predict(X_train_transformed),
    rf_model.predict(X_train_transformed),
    bag_default.predict(X_train_transformed),
    bag_tuned.predict(X_train_transformed),
    xgb_model.predict(X_train_transformed),
    cat_model.predict(X_train_transformed)
])

print("Meta-features shape:", meta_features.shape)
meta_model = LinearRegression()
meta_model.fit(meta_features, y_train_series)
y_pred_stack = meta_model.predict(meta_features)
rmse = np.sqrt(mean_squared_error(y_train_series, y_pred_stack))
mae = mean_absolute_error(y_train_series, y_pred_stack)
r2 = r2_score(y_train_series, y_pred_stack)
mape = np.mean(np.abs((y_train_series - y_pred_stack) / (y_train_series + 1e-8))) * 100

print("\n" + "="*50)
print("STACKING META-MODEL — Training Results")
print("="*50)
print(f"RMSE : {rmse:.4f}")
print(f"MAE  : {mae:.4f}")
print(f"MAPE : {mape:.2f}%")
print(f"R²   : {r2:.4f}")
print("="*50)

Loading base models...
All base models loaded successfully.
Generating meta-features...
Meta-features shape: (300658, 6)

STACKING META-MODEL — Training Results
RMSE : 0.0916
MAE  : 0.0353
MAPE : 0.65%
R²   : 0.9672


In [50]:
save_path = os.path.join(MODEL_PATH, "stacking_model.pkl")
with open(save_path, "wb") as f:
    pickle.dump(meta_model, f)

print(f"\nStacking model saved to: {save_path}")


Stacking model saved to: /content/drive/MyDrive/ML2 regression/artifacts/models/stacking_model.pkl


| Model               | RMSE ↓  | MAE ↓   | MAPE ↓ | R² ↑   | Behavior                                      |
|---------------------|---------|---------|--------|--------|-----------------------------------------------|
| Decision Tree       | 0.3570  | 0.2129  | 4.03%  | 0.5018 | Weak learner, high bias                       |
| Random Forest       | 0.2916  | 0.1648  | 3.34%  | 0.6676 | Moderate learner, stable but not strong       |
| Bagging (Tuned)     | 0.2598  | 0.1449  | 2.80%  | 0.7362 | Balanced, medium variance                     |
| CatBoost            | 0.1738  | 0.0971  | 1.71%  | 0.8819 | Strong gradient model                         |
| XGBoost             | 0.2004  | 0.1071  | 2.00%  | 0.8430 | Strong gradient model                         |
| Bagging (Default)   | 0.0978  | 0.0314  | 0.61%  | 0.9626 | Extremely high variance (overfits heavily)    |
| STACKING META‑MODEL | 0.0916  | 0.0353  | 0.65%  | 0.9672 | Best overall — combines strengths of all models |


## 📘 Notebook Overview: Model Retraining with Best Parameters

In this notebook, we rebuilt the entire machine‑learning model suite using a clean and reproducible workflow.  
All models were trained **only on the training set**, using the **best hyperparameters** discovered earlier.  
No validation or test data was used here.

### What We Did
- Loaded all preprocessing artifacts (encoders, transformers, and transformed matrices).
- Retrained each model **without hyperparameter search**, using the previously selected best parameters:
  - Decision Tree  
  - Random Forest  
  - Bagging (Default)  
  - Bagging (Tuned)  
  - XGBoost  
  - CatBoost  
- Saved every trained model into the project’s `artifacts/models` directory.
- Generated predictions on the training set and computed core metrics (RMSE, MAE, MAPE, R²).
- Built a **Stacking Meta‑Model** by:
  - Loading all base models  
  - Generating meta‑features from their predictions  
  - Training a Linear Regression meta‑learner  
  - Saving the final stacking model  

### Purpose of This Notebook
This notebook prepares a complete set of finalized, retrained models that will later be evaluated on the **validation set** in the next notebook. It ensures consistency, reproducibility, and a clean separation between training, validation, and testing stages.
